In [ ]:
// 📍 Definir la región de interés (Sogamoso, Boyacá, Colombia)
var sogamoso = ee.Geometry.Point([-72.9286, 5.7143]);

// 🔹 FILTRAR IMAGEN SENTINEL-2 (Óptico) - Nivel 2A con corrección atmosférica
var sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR")
                  .filterBounds(sogamoso)
                  .filterDate('2023-01-01', '2023-12-31')
                  .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
                  .sort('CLOUDY_PIXEL_PERCENTAGE')
                  .first(); // Seleccionar la imagen con menos nubes

print("Imagen Sentinel-2 seleccionada:", sentinel2);
print("Bandas Sentinel-2:", sentinel2.bandNames());

// 🔹 FILTRAR IMAGEN SENTINEL-1 (Radar) - Polarización VV y VH
var sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD")
                  .filterBounds(sogamoso)
                  .filterDate('2023-01-01', '2023-12-31')
                  .filterMetadata('instrumentMode', 'equals', 'IW')
                  .filterMetadata('orbitProperties_pass', 'equals', 'DESCENDING')
                  .filterMetadata('resolution_meters', 'equals', 10)
                  .first(); // Tomar la primera imagen disponible

print("Imagen Sentinel-1 seleccionada:", sentinel1);
print("Bandas Sentinel-1:", sentinel1.bandNames());

// 📌 VISUALIZACIÓN
// Sentinel-2 (Óptico) - RGB Natural
var visS2 = {bands: ['B4', 'B3', 'B2'], min: 0, max: 3000, gamma: 1.4};
Map.centerObject(sogamoso, 10);
Map.addLayer(sentinel2, visS2, "Sentinel-2 (Óptico)");

// Sentinel-1 (Radar) - Polarización VV
var visS1 = {bands: ['VV'], min: -25, max: 0, gamma: 1};
Map.addLayer(sentinel1, visS1, "Sentinel-1 (Radar VV)");

// 📤 EXPORTAR SENTINEL-2 A GOOGLE DRIVE
Export.image.toDrive({
  image: sentinel2.select(['B4', 'B3', 'B2', 'B8']), // Rojo, Verde, Azul, Infrarrojo Cercano
  description: "Sentinel2_Sogamoso",
  folder: "GEE_Exports",
  fileNamePrefix: "Sentinel2_Sogamoso",
  region: sogamoso.buffer(5000).bounds(),
  scale: 10,
  maxPixels: 1e13,
  fileFormat: "GeoTIFF"
});

// 📤 EXPORTAR SENTINEL-1 A GOOGLE DRIVE
Export.image.toDrive({
  image: sentinel1.select(['VV', 'VH']), // Polarizaciones VV y VH
  description: "Sentinel1_Sogamoso",
  folder: "GEE_Exports",
  fileNamePrefix: "Sentinel1_Sogamoso",
  region: sogamoso.buffer(5000).bounds(),
  scale: 10,
  maxPixels: 1e13,
  fileFormat: "GeoTIFF"
});
